<a href="https://colab.research.google.com/github/Arpon-30/CVPR/blob/main/Final/mnist%20_digit%20_detection/digit_cam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# Load model
model = load_model(r"model.h5")
print("✅ Model loaded")

# Open camera
cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)

if not cap.isOpened():
    print("❌ Camera not opened")
    exit()

print("✅ Camera running — Press q to quit | r to reset")

roi_size = 200

# AUTO CAPTURE SETTINGS
CONF_THRESHOLD = 80      # confidence %
STABLE_FRAMES = 10       # consecutive frames required

stable_count = 0
captured = False
final_digit = None
final_conf = None

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    h, w = gray.shape

    # Central ROI
    x1, y1 = w//2 - roi_size//2, h//2 - roi_size//2
    x2, y2 = x1 + roi_size, y1 + roi_size
    roi = gray[y1:y2, x1:x2]

    # Draw ROI box
    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

    if not captured:
        # Preprocessing
        roi_blur = cv2.GaussianBlur(roi, (5, 5), 0)
        roi_thresh = cv2.adaptiveThreshold(
            roi_blur, 255,
            cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
            cv2.THRESH_BINARY_INV,
            11, 2
        )

        contours, _ = cv2.findContours(
            roi_thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
        )

        if contours:
            c = max(contours, key=cv2.contourArea)
            x, y, w1, h1 = cv2.boundingRect(c)

            if w1 * h1 > 500:
                digit_roi = roi_thresh[y:y+h1, x:x+w1]

                # MNIST formatting
                digit_roi = cv2.resize(digit_roi, (20, 20))
                canvas = np.zeros((28, 28), dtype=np.uint8)
                canvas[4:24, 4:24] = digit_roi

                canvas = canvas / 255.0
                canvas = canvas.reshape(1, 784)

                prediction = model.predict(canvas, verbose=0)
                digit = np.argmax(prediction)
                confidence = np.max(prediction) * 100

                # Stability check
                if confidence >= CONF_THRESHOLD:
                    stable_count += 1
                else:
                    stable_count = 0

                if stable_count >= STABLE_FRAMES:
                    captured = True
                    final_digit = digit
                    final_conf = confidence

                # Overlay text on SAME webcam window
                cv2.putText(frame,
                            f"Digit: {digit} ({confidence:.2f}%)",
                            (20, 50),
                            cv2.FONT_HERSHEY_SIMPLEX,
                            1.2, (255, 0, 0), 3)

                cv2.putText(frame,
                            f"Stability: {stable_count}/{STABLE_FRAMES}",
                            (20, 90),
                            cv2.FONT_HERSHEY_SIMPLEX,
                            0.8, (0, 255, 255), 2)

    else:
        cv2.putText(frame,
                    f"AUTO CAPTURED: {final_digit} ({final_conf:.2f}%)",
                    (20, 60),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    1.4, (0, 0, 255), 4)

        cv2.putText(frame,
                    "Press 'r' to reset",
                    (20, 110),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.9, (0, 255, 0), 2)

    # ✅ ONLY ONE WINDOW
    cv2.imshow("MNIST Digit Detection", frame)

    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break
    elif key == ord('r'):
        captured = False
        stable_count = 0
        final_digit = None
        final_conf = None

cap.release()
cv2.destroyAllWindows()


✅ Model loaded
✅ Camera running — Press q to quit | r to reset
